In [45]:
import pandas as pd
import numpy as np
from datetime import timedelta

import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 
from scripts.indicators import psar,zigzag,polyfit_trend, resample




C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [46]:
klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #df.set_index('datetime',inplace=True)

df = resample(df,(60*24))

#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
#df = df[500:1500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


In [61]:
df['price'] = df['close']
df['zero'] = 0.0

#df['ma'] = df['close'].rolling(window=21).mean()

df['mm'] = df['close'].ewm(span=7).mean()
df['mm'] = df['close'].rolling(window=5).mean()
#df['mm'] = df['close']*0.20+df['close'].shift(1)*.80


df['mm_h'] = df['high'].rolling(window=5).mean()
df['mm_l'] = df['low'].rolling(window=5).mean()
df = polyfit_trend(df,column='close',window=7,fwd=1,prefix='c')
df = polyfit_trend(df,column='high' ,window=7,fwd=1,prefix='h')
df = polyfit_trend(df,column='low'  ,window=7,fwd=1,prefix='l')

df['buy']  = np.where((df['c_pred']>df['h_pred']),df['high'],None)
df['sell'] = np.where((df['c_pred']<df['l_pred']),df['low'],None)

In [62]:
if 'volume' in df.columns:
	df.drop(columns=['volume'],inplace=True)
to_show = df[200:600]
indicators = [
     {'col': 'mm_h','name': 'h_pred','color': 'gray','row': 1, 'mode':'lines',},
     {'col': 'mm_l','name': 'h_pred','color': 'gray','row': 1, 'mode':'lines',},
     {'col': 'h_pred','name': 'h_pred','color': 'green','row': 1, 'mode':'lines',},
     {'col': 'l_pred','name': 'h_pred','color': 'red','row': 1, 'mode':'lines',},
     #{'col': 'mm','name': 'mm','color': 'white','row': 1, 'mode':'lines',},
     #{'col': 'ma','name': 'ma','color': 'orange','row': 1, 'mode':'lines',},
     #  {'col': 'trend+','name': 'trend+','color': 'green','row': 1, 'mode':'markers',},
     #  {'col': 'trend-','name': 'trend-','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'st_trigger','name': 'ST Trigger','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'st_high','name': 'ST High','color': 'green','row': 1, 'mode':'markers',},
        ]

indicators_out = [
     #{'col': 'h_change','name': 'h_change','color': 'green','row': 2, 'mode':'lines',},
     #{'col': 'l_change','name': 'l_change','color': 'red','row': 2, 'mode':'lines',},
     #{'col': 'zero','name': 'zero','color': 'gray','row': 2, 'mode':'lines',},
     #{'col': 'pf_trend','name': 'trend','color': 'yellow','row': 3, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
        {'df':to_show,'col':'sell','name': 'SELL','color': 'red','symbol': 'circle-dot' }, 
        {'df':to_show,'col':'buy','name': 'BUY','color': 'green','symbol': 'circle-dot' }, 
     ]

fig = fn.ohlc_chart(to_show,indicators=indicators,indicators_out = indicators_out, events=events) #
fig.show()